In [5]:
import time
import json
from transformers import pipeline

# Initialize the text generation pipeline with your chosen model (e.g., "gpt2").
generator = pipeline("text-generation", model="gpt2")

# =============================================================================
# 1. Define Placeholders for Inputs and Target Fields
# =============================================================================
# For inputs, we simply return placeholders rather than random values.
placeholders_inputs = {
    1: {"Plan Name": "{Plan Name}"},
    2: {"Premium Type": "{Premium Type}"},
    3: {"Plan Name": "{Plan Name}", "Premium Type": "{Premium Type}"},
    4: {"Plan Name": "{Plan Name}", "Premium Type": "{Premium Type}", "Disease": "{Disease}"},
    5: {"Plan Name": "{Plan Name}", "Premium Type": "{Premium Type}", "Disease": "{Disease}", "Amount": "{Amount}"}
}

# Define target fields as placeholders in curly braces.
target_fields = {
    1: "{Premium Type}, {DisCount}, {Diseases}, {CoverageLevel}, {PlanFocus}, {Advantage}",
    2: "{Plan Name}, {Deductible}, {Co‑pay Percentage}, {Plan Term}, {Tax Redemption}, {Plan Count}, {Monthly Payment}, {Advantage}",
    3: "{Monthly Payment}, {Deductible}, {Co‑pay Percentage}, {Plan Term}, {Tax Redemption}, {Benefits}",
    4: "{Maximum Coverage}, {Deductible}, {Co‑pay Percentage}, {Waiting Period}, {Claims Settled}, {Renewability}, {Hospital Coverage}, {Benefits}, {Tax Redemption}",
    5: "{Calculation}, {Maximum Coverage}, {Deductible}, {Co‑pay Percentage}, {Waiting Period}, {Claims Settled}, {Renewability}, {Hospital Coverage}, {Benefits}, {Tax Redemption}"
}

# =============================================================================
# 2. Define the Master Prompt Template
# =============================================================================
MASTER_PROMPT = """
You are an expert insurance data transformer. Given the following details, generate a unique and comprehensive insurance template that fully covers all required fields and edge cases.

Allowed Inputs (placeholders):
- Plan Names: {Plan Names}
- Premium Types: {Premium Types}
- Diseases: {Diseases}
- Amount: {Amount}

Levels and Target Fields (placeholders):
- Level 1 (Input: {Plan Name}): Targets – {Targets1}
- Level 2 (Input: {Premium Type}): Targets – {Targets2}
- Level 3 (Input: {Plan Name}, {Premium Type}): Targets – {Targets3}
- Level 4 (Input: {Plan Name}, {Disease}, {Premium Type}): Targets – {Targets4}
- Level 5 (Input: {Plan Name}, {Disease}, {Premium Type}, {Amount}): Targets – {Targets5}

Task:
Based on the selected level and its placeholder inputs, generate one unique, natural language template.
The generated template should:
- Clearly reference the input placeholders (e.g., "{Plan Name}", "{Premium Type}", etc.).
- List the corresponding target columns as placeholders.
- Use varied natural language while keeping all field names in curly braces.

Generate one unique template that meets these requirements.
"""

# Format the master prompt with placeholder values.
# Added both singular and plural keys to satisfy all placeholders in MASTER_PROMPT.
formatted_master_prompt = MASTER_PROMPT.format(
    **{
        "Plan Names": "{Plan Name}",
        "Plan Name": "{Plan Name}",
        "Premium Types": "{Premium Type}",
        "Premium Type": "{Premium Type}",
        "Diseases": "{Disease}",
        "Disease": "{Disease}",
        "Amount": "{Amount}",
        "Targets1": target_fields[1],
        "Targets2": target_fields[2],
        "Targets3": target_fields[3],
        "Targets4": target_fields[4],
        "Targets5": target_fields[5]
    }
)

# =============================================================================
# 3. Functions for Placeholder Input Generation and Prompt Building
# =============================================================================
def get_placeholder_inputs_for_level(level):
    """
    Return the placeholder inputs for a given level.
    """
    return placeholders_inputs.get(level, {})

def build_prompt(level, inputs):
    """
    Build a custom prompt for the text generation API call by appending the selected level and its placeholder inputs.
    """
    prompt = f"{formatted_master_prompt}\n\n"
    prompt += f"Selected Level: {level}\n"
    prompt += "Inputs:\n"
    for key, value in inputs.items():
        prompt += f"- {key}: {value}\n"
    prompt += "\nGenerate a unique template based on these placeholder inputs."
    return prompt

def generate_template(prompt):
    """
    Use the Hugging Face text-generation pipeline to generate a template from the prompt.
    We use max_new_tokens to generate new tokens beyond the prompt.
    """
    try:
        generated = generator(prompt, do_sample=True, temperature=0.8, max_new_tokens=100, truncation=True)
        generated_text = generated[0]['generated_text']
        return generated_text.strip()
    except Exception as e:
        print("Hugging Face generation error:", e)
        return None

# =============================================================================
# 4. Main Template Generation Loop (Generate 10 Templates)
# =============================================================================
def main(num_templates=10, output_file="generated_templates.json"):
    results = []
    for i in range(num_templates):
        # Randomly choose a level (1 to 5)
        import random  # Ensure random is imported
        level = random.randint(1, 5)
        # Get placeholder inputs for the level
        inputs = get_placeholder_inputs_for_level(level)
        # Build the final prompt for this API call
        input_prompt = build_prompt(level, inputs)
        # Generate the template via the Hugging Face pipeline
        output_template = generate_template(input_prompt)
        
        if output_template:
            entry = {
                "Level": level,
                "Input Prompt": input_prompt,
                "Inputs": inputs,
                "Target": target_fields[level],
                "Output Template": output_template
            }
            results.append(entry)
            # Print the generated details in real time
            print(f"\n--- Template {i + 1} (Level {level}) ---")
            print("Input Prompt:")
            print(input_prompt)
            print("\nInputs:")
            print(inputs)
            print("\nTarget Columns:")
            print(target_fields[level])
            print("\nOutput Template:")
            print(output_template)
        else:
            print(f"Failed to generate template {i + 1}")
        
        # Optional: Sleep to manage rate limits
        time.sleep(1)
    
    # Save all results to a JSON file
    with open(output_file, "w") as f:
        json.dump(results, f, indent=2)
    print(f"\nSaved {len(results)} templates to {output_file}")

if __name__ == "__main__":
    main(10)


Device set to use cpu
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



--- Template 1 (Level 1) ---
Input Prompt:

You are an expert insurance data transformer. Given the following details, generate a unique and comprehensive insurance template that fully covers all required fields and edge cases.

Allowed Inputs (placeholders):
- Plan Names: {Plan Name}
- Premium Types: {Premium Type}
- Diseases: {Disease}
- Amount: {Amount}

Levels and Target Fields (placeholders):
- Level 1 (Input: {Plan Name}): Targets – {Premium Type}, {DisCount}, {Diseases}, {CoverageLevel}, {PlanFocus}, {Advantage}
- Level 2 (Input: {Premium Type}): Targets – {Plan Name}, {Deductible}, {Co‑pay Percentage}, {Plan Term}, {Tax Redemption}, {Plan Count}, {Monthly Payment}, {Advantage}
- Level 3 (Input: {Plan Name}, {Premium Type}): Targets – {Monthly Payment}, {Deductible}, {Co‑pay Percentage}, {Plan Term}, {Tax Redemption}, {Benefits}
- Level 4 (Input: {Plan Name}, {Disease}, {Premium Type}): Targets – {Maximum Coverage}, {Deductible}, {Co‑pay Percentage}, {Waiting Period}, {Claims S

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



--- Template 2 (Level 4) ---
Input Prompt:

You are an expert insurance data transformer. Given the following details, generate a unique and comprehensive insurance template that fully covers all required fields and edge cases.

Allowed Inputs (placeholders):
- Plan Names: {Plan Name}
- Premium Types: {Premium Type}
- Diseases: {Disease}
- Amount: {Amount}

Levels and Target Fields (placeholders):
- Level 1 (Input: {Plan Name}): Targets – {Premium Type}, {DisCount}, {Diseases}, {CoverageLevel}, {PlanFocus}, {Advantage}
- Level 2 (Input: {Premium Type}): Targets – {Plan Name}, {Deductible}, {Co‑pay Percentage}, {Plan Term}, {Tax Redemption}, {Plan Count}, {Monthly Payment}, {Advantage}
- Level 3 (Input: {Plan Name}, {Premium Type}): Targets – {Monthly Payment}, {Deductible}, {Co‑pay Percentage}, {Plan Term}, {Tax Redemption}, {Benefits}
- Level 4 (Input: {Plan Name}, {Disease}, {Premium Type}): Targets – {Maximum Coverage}, {Deductible}, {Co‑pay Percentage}, {Waiting Period}, {Claims S

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...


--- Template 8 (Level 4) ---
Input Prompt:

You are an expert insurance data transformer. Given the following details, generate a unique and comprehensive insurance template that fully covers all required fields and edge cases.

Allowed Inputs (placeholders):
- Plan Names: {Plan Name}
- Premium Types: {Premium Type}
- Diseases: {Disease}
- Amount: {Amount}

Levels and Target Fields (placeholders):
- Level 1 (Input: {Plan Name}): Targets – {Premium Type}, {DisCount}, {Diseases}, {CoverageLevel}, {PlanFocus}, {Advantage}
- Level 2 (Input: {Premium Type}): Targets – {Plan Name}, {Deductible}, {Co‑pay Percentage}, {Plan Term}, {Tax Redemption}, {Plan Count}, {Monthly Payment}, {Advantage}
- Level 3 (Input: {Plan Name}, {Premium Type}): Targets – {Monthly Payment}, {Deductible}, {Co‑pay Percentage}, {Plan Term}, {Tax Redemption}, {Benefits}
- Level 4 (Input: {Plan Name}, {Disease}, {Premium Type}): Targets – {Maximum Coverage}, {Deductible}, {Co‑pay Percentage}, {Waiting Period}, {Claims S

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



--- Template 9 (Level 5) ---
Input Prompt:

You are an expert insurance data transformer. Given the following details, generate a unique and comprehensive insurance template that fully covers all required fields and edge cases.

Allowed Inputs (placeholders):
- Plan Names: {Plan Name}
- Premium Types: {Premium Type}
- Diseases: {Disease}
- Amount: {Amount}

Levels and Target Fields (placeholders):
- Level 1 (Input: {Plan Name}): Targets – {Premium Type}, {DisCount}, {Diseases}, {CoverageLevel}, {PlanFocus}, {Advantage}
- Level 2 (Input: {Premium Type}): Targets – {Plan Name}, {Deductible}, {Co‑pay Percentage}, {Plan Term}, {Tax Redemption}, {Plan Count}, {Monthly Payment}, {Advantage}
- Level 3 (Input: {Plan Name}, {Premium Type}): Targets – {Monthly Payment}, {Deductible}, {Co‑pay Percentage}, {Plan Term}, {Tax Redemption}, {Benefits}
- Level 4 (Input: {Plan Name}, {Disease}, {Premium Type}): Targets – {Maximum Coverage}, {Deductible}, {Co‑pay Percentage}, {Waiting Period}, {Claims S

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



--- Template 10 (Level 5) ---
Input Prompt:

You are an expert insurance data transformer. Given the following details, generate a unique and comprehensive insurance template that fully covers all required fields and edge cases.

Allowed Inputs (placeholders):
- Plan Names: {Plan Name}
- Premium Types: {Premium Type}
- Diseases: {Disease}
- Amount: {Amount}

Levels and Target Fields (placeholders):
- Level 1 (Input: {Plan Name}): Targets – {Premium Type}, {DisCount}, {Diseases}, {CoverageLevel}, {PlanFocus}, {Advantage}
- Level 2 (Input: {Premium Type}): Targets – {Plan Name}, {Deductible}, {Co‑pay Percentage}, {Plan Term}, {Tax Redemption}, {Plan Count}, {Monthly Payment}, {Advantage}
- Level 3 (Input: {Plan Name}, {Premium Type}): Targets – {Monthly Payment}, {Deductible}, {Co‑pay Percentage}, {Plan Term}, {Tax Redemption}, {Benefits}
- Level 4 (Input: {Plan Name}, {Disease}, {Premium Type}): Targets – {Maximum Coverage}, {Deductible}, {Co‑pay Percentage}, {Waiting Period}, {Claims 